In [1]:
import requests
import pandas as pd
import time

In [ ]:
# Create empty dataframe
col_names = ['gameID', 'date', 'homeTeamScore', 'visitorTeamScore', 
             'season', 'period', 'status', 'time', 'postseasonBoolean', 
             'homeTeamID', 'homeTeamAbbrev', 'homeTeamCity', 'homeTeamConference', 
             'homeTeamDivision', 'homeTeamFullName', 'homeTeamName', 'visitorTeamID', 
             'visitorTeamAbbrev', 'visitorTeamCity', 'visitorTeamConference', 
             'visitorTeamDivision', 'visitorTeamFullName', 'visitorTeamName']
games_df = pd.DataFrame(columns=col_names)


In [ ]:
# Get games for last 5 years
years = [2014, 2015, 2016, 2017, 2018]
for y in years:
    
    games_url_initial = f'https://www.balldontlie.io/api/v1/games?seasons[]={y}'
    data_initial = requests.get(games_url_initial).json()
    
    for page in range(1, data_initial['meta']['total_pages']+1):
        
        games_url = f'https://www.balldontlie.io/api/v1/games?seasons[]={y}&page={page}'
        data = requests.get(games_url).json()
        
        time.sleep(1)

        for game in data['data']:
            games_df.loc[len(games_df.index)] = [ game['id'], game['date'], game['home_team_score'], 
                                                 game['visitor_team_score'], game['season'], game['period'], 
                                                 game['status'], game['time'], game['postseason'], game['home_team']['id'],
                                                 game['home_team']['abbreviation'], game['home_team']['city'], game['home_team']['conference'], 
                                                 game['home_team']['division'], game['home_team']['full_name'], game['home_team']['name'], 
                                                 game['visitor_team']['id'], game['visitor_team']['abbreviation'], 
                                                 game['visitor_team']['city'], game['visitor_team']['conference'], 
                                                 game['visitor_team']['division'], game['visitor_team']['full_name'], 
                                                 game['visitor_team']['name'] ]

        
games_df.head(25)


In [ ]:
games_df.shape

In [ ]:
games_df.to_csv('allGamesFinal.csv')

## GAME STAT DATA

In [ ]:
# Get player data for each player in all of the games in the dataframe above
col_names2 = ["gameID", "statsID", "assists", "blocks", "defensiveRebounds", "threePointFieldGoalPercent", 
              "threePointFieldGoalAttempted", "threePointFieldGoal", "fieldGoalPercent", "fieldGoalAttempt", 
              "fieldGoalMade", "freeThrowPercent", "freeThrowAttempt", 
              "freeThrowMade", "minutes", "offensiveRebound", "personalFouls", "playerID", "playerFirstName", 
              "playerHeightFeet", "playerHeightInches", "playerLastName", 
              "playerPosition", "team_id", "playerWeightPounds", "points", 
              "rebounds", "steals", "teamID", "turnovers"]

game_stats_df = pd.DataFrame(columns=col_names2)


In [ ]:
game_stats_df.shape
# len(game_id_list)

In [ ]:
# Gather data from API
game_id_list = games_df['gameID']

for id in game_id_list[6444:]:
    print(f'Game id: {id}')
    #player_url_initial = f'https://www.balldontlie.io/api/v1/stats?game_ids[]={id}&per_page=100'
    #player_data_initial = requests.get(player_url_initial).json()
    
    for page in range(1,2):
        print(f'Page: {page}')
        player_games_url = f'https://www.balldontlie.io/api/v1/stats?game_ids[]={id}&page={page}&per_page=100'
        player_data = requests.get(player_games_url).json()
        print(len(player_data['data']))
        
        time.sleep(1)

        for stat in player_data['data']:
            game_stats_df.loc[len(game_stats_df.index)] = [ id, stat['id'], stat['ast'], stat['blk'],
                                                          stat['dreb'], stat['fg3_pct'], stat['fg3a'],
                                                          stat['fg3m'], stat['fg_pct'], stat['fga'],
                                                          stat['fgm'], stat['ft_pct'], stat['fta'],
                                                          stat['ftm'], stat['min'], stat['oreb'],
                                                          stat['pf'], stat['player']['id'], stat['player']['first_name'],
                                                          stat['player']['height_feet'], stat['player']['height_inches'],
                                                          stat['player']['last_name'], stat['player']['position'],
                                                          stat['player']['team_id'], stat['player']['weight_pounds'],
                                                          stat['pts'], stat['reb'], stat['stl'], stat['team']['id'],
                                                          stat['turnover'] ]
        print(f'Finished stats on page {page}')

print(f'Shape: {game_stats_df.shape}')  
game_stats_df.head(50)

In [ ]:
#48787, 49061, 48826, 48835, 49071, 48801, 48809 - incomplete data due to weird exception
#game_stats_df.tail(25)
game_stats_df.shape


In [ ]:
game_stats_df.to_csv('allStatsFinal.csv')


# MORE ETL

## Add winner/loser column

## Combine all relevant data to one master dataframe

In [2]:
# Load existing CSVs if necessary:
game_df = pd.read_csv('allGamesFinal.csv')
game_stats_df = pd.read_csv('allStatsFinal.csv')

game_stats_df.head()

,Unnamed: 0,gameID,statsID,assists,blocks,defensiveRebounds,threePointFieldGoalPercent,threePointFieldGoalAttempted,threePointFieldGoal,fieldGoalPercent,...,playerHeightInches,playerLastName,playerPosition,team_id,playerWeightPounds,points,rebounds,steals,teamID,turnovers
0,0,27276,708256,1.0,0.0,4.0,0.000,3.0,0.0,0.385,...,10.0,Parsons,F,1,230.0,11.0,4.0,0.0,7,1.0
1,1,27276,708257,1.0,0.0,3.0,0.286,7.0,2.0,0.353,...,0.0,Nowitzki,F,7,245.0,17.0,3.0,0.0,7,2.0
2,2,27276,708258,2.0,3.0,2.0,0.000,0.0,0.0,0.000,...,1.0,Chandler,C,11,240.0,4.0,3.0,0.0,7,1.0
3,3,27276,708259,1.0,0.0,3.0,0.000,1.0,0.0,0.462,...,NaN,Ellis,NaN,10,NaN,14.0,3.0,1.0,7,2.0
4,4,27276,708260,1.0,0.0,3.0,0.400,5.0,2.0,0.222,...,NaN,Nelson,NaN,22,NaN,6.0,4.0,0.0,7,1.0


In [7]:
# <expression1> if <condition> else <expression2>
#df['Discounted_Price'] = df.apply(lambda row: row.Cost - (row.Cost * 0.1), axis = 1) 

# Create a winners column
game_df['winner'] = game_df.apply(lambda row: row.homeTeamID if row.homeTeamScore > row.visitorTeamScore else row.visitorTeamID, axis=1)

,Unnamed: 0,gameID,date,homeTeamScore,visitorTeamScore,season,period,status,time,postseasonBoolean,...,homeTeamFullName,homeTeamName,visitorTeamID,visitorTeamAbbrev,visitorTeamCity,visitorTeamConference,visitorTeamDivision,visitorTeamFullName,visitorTeamName,winner
0,0,27276,2014-11-06T00:00:00.000Z,108,87,2014,4,Final,,False,...,Portland Trail Blazers,Trail Blazers,7,DAL,Dallas,West,Southwest,Dallas Mavericks,Mavericks,25
1,1,27277,2014-11-07T00:00:00.000Z,122,119,2014,4,Final,,False,...,Charlotte Hornets,Hornets,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,4
2,2,27278,2014-11-08T00:00:00.000Z,87,98,2014,4,Final,,False,...,Houston Rockets,Rockets,10,GSW,Golden State,West,Pacific,Golden State Warriors,Warriors,10
3,3,27279,2014-11-09T00:00:00.000Z,116,100,2014,4,Final,,False,...,Portland Trail Blazers,Trail Blazers,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets,25
4,4,27280,2014-11-10T00:00:00.000Z,85,89,2014,4,Final,,False,...,LA Clippers,Clippers,27,SAS,San Antonio,West,Southwest,San Antonio Spurs,Spurs,27
5,5,27281,2014-11-11T00:00:00.000Z,107,102,2014,4,Final,,False,...,Memphis Grizzlies,Grizzlies,14,LAL,Los Angeles,West,Pacific,Los Angeles Lakers,Lakers,15
6,6,27282,2014-11-12T00:00:00.000Z,107,103,2014,4,Final,,False,...,Washington Wizards,Wizards,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,30
7,7,27283,2014-11-12T00:00:00.000Z,101,113,2014,4,Final,,False,...,Minnesota Timberwolves,Timberwolves,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets,11
8,8,27284,2014-11-13T00:00:00.000Z,93,100,2014,4,Final,,False,...,Toronto Raptors,Raptors,5,CHI,Chicago,East,Central,Chicago Bulls,Bulls,5
9,9,27285,2014-11-14T00:00:00.000Z,121,122,2014,4,Final,,False,...,Boston Celtics,Celtics,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,6


In [8]:
game_df.to_csv('allGamesFinalWithWinnerCol.csv')

In [21]:
# winner_val_for_id = game_df.loc[ game_df['gameID'] == game_stats_df.loc[0]['gameID'] ]['winner']
# winner_val_for_id.values[0]


25

In [22]:
# if row.teamID == game_df.loc[game_df['gameID'] == row.teamID]['winner']
game_stats_df['winner'] = game_stats_df.apply(lambda row: 1 if row.teamID == game_df.loc[game_df['gameID'] == row.gameID]['winner'].values[0] else 0, axis=1)



In [24]:
game_stats_df.head(50)

,Unnamed: 0,gameID,statsID,assists,blocks,defensiveRebounds,threePointFieldGoalPercent,threePointFieldGoalAttempted,threePointFieldGoal,fieldGoalPercent,...,playerLastName,playerPosition,team_id,playerWeightPounds,points,rebounds,steals,teamID,turnovers,winner
0,0,27276,708256,1.0,0.0,4.0,0.000,3.0,0.0,0.385,...,Parsons,F,1,230.0,11.0,4.0,0.0,7,1.0,0
1,1,27276,708257,1.0,0.0,3.0,0.286,7.0,2.0,0.353,...,Nowitzki,F,7,245.0,17.0,3.0,0.0,7,2.0,0
2,2,27276,708258,2.0,3.0,2.0,0.000,0.0,0.0,0.000,...,Chandler,C,11,240.0,4.0,3.0,0.0,7,1.0,0
3,3,27276,708259,1.0,0.0,3.0,0.000,1.0,0.0,0.462,...,Ellis,NaN,10,NaN,14.0,3.0,1.0,7,2.0,0
4,4,27276,708260,1.0,0.0,3.0,0.400,5.0,2.0,0.222,...,Nelson,NaN,22,NaN,6.0,4.0,0.0,7,1.0,0
5,5,27276,708261,4.0,0.0,2.0,0.000,2.0,0.0,0.286,...,Harris,G,7,185.0,7.0,3.0,1.0,7,0.0,0
6,6,27276,708262,1.0,0.0,1.0,0.000,0.0,0.0,0.600,...,Wright,NaN,10,NaN,6.0,3.0,0.0,7,0.0,0
7,7,27276,708263,0.0,1.0,3.0,1.000,1.0,1.0,0.500,...,Aminu,F,22,220.0,7.0,5.0,2.0,7,0.0,0
8,8,27276,708264,2.0,0.0,2.0,0.000,2.0,0.0,0.375,...,Barea,G,7,185.0,8.0,2.0,2.0,7,0.0,0
9,9,27276,708265,0.0,0.0,1.0,0.000,1.0,0.0,0.200,...,Jefferson,NaN,3,NaN,3.0,2.0,0.0,7,0.0,0


In [25]:
game_stats_df.to_csv('allStatsFinalWithWinnerCol.csv')